<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# lies die Zeilen der datei links.txt aus und gib die länge der liste aus

with open("links.txt", 'r') as f:
    links = [line.strip() for line in f.readlines()] #strip() entfernt \n 

print(links[0])

https://finance.yahoo.com/news/1-singapores-shein-holds-talks-120057806.html


In [10]:
from bs4 import BeautifulSoup
import requests

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

import sqlite3

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

i = 0
for link in links:
    link = link.strip()
    res = requests.get(link)

    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.find("title").text

    if(soup == None or title == 'Yahoo'):
        continue

    print(title)

    soup = soup.find("div", attrs={'class':'caas-body'})

    for div in soup.find_all('div'):
        div.unwrap()

    for ul in soup.find_all('ul'):
        # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
        p_tag = ul.find_previous_sibling('p')

        # Wenn ein <p>-Tag gefunden wurde, entferne es
        if p_tag:
            p_tag.decompose()

    # Entferne alle <div>-Elemente
    for ul in soup.find_all("ul"):
        ul.decompose()

    for button in soup.find_all("button"):
        button.decompose()

    for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
        p_tag.decompose()

    # Dein Text
    text = ""

    for p in soup.find_all('p'):
        text = text + p.text + "\n"
    # Tokenisiere den Text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Wende das Modell an
    with torch.no_grad():
        outputs = model(**inputs)

    # Hole die Modellvorhersagen
    predictions = outputs.logits

    # Konvertiere Vorhersagen in Wahrscheinlichkeiten und hole das maximale
    predictions = torch.nn.functional.softmax(predictions, dim=1)
    predicted_class = torch.argmax(predictions).item()

    # Zuordnung der Klassen
    classes = ["bearish", "neutral", "bullish"]

    #wenn das sentiment bullish ist, dann nicht speichern (bullish ist schon über 50%)
    if classes[predicted_class] == 'bullish':
        print("\033[91mbullish\033[0m")
        continue

    # Ergebnis
    print(f"Sentiment: {classes[predicted_class]}")

    # speichere das ergebnis in einer datenbank
    conn = sqlite3.connect('sentiment.db')
    c = conn.cursor()

    # Create table with title, link, text, sentiment
    c.execute('''CREATE TABLE IF NOT EXISTS sentiment
                (title text, link text, text text, sentiment text)''')
    
    # Before inserting a row, check if it already exists by checking the title
    c.execute("SELECT * FROM sentiment WHERE title=?", (title,))

    exists = c.fetchone()[0]

    if exists:
        print("Already in database")
        continue

    # Insert a row of data
    c.execute("INSERT INTO sentiment VALUES (?, ?, ?, ?)", (title, link, text, classes[predicted_class]))
    
    conn.commit()
    conn.close()
    i = i + 1

print(f"Added {i} new articles to database")

UPDATE 1-Singapore's Shein holds talks with LSE on possible listing -Sky News
Sentiment: bearish
Already in database
Brazil Economists Lift Growth Forecasts as Activity Outperforms
bullish
Sovereign bond investors on hold ahead of U.S. data, central bank meetings
Sentiment: bearish
Already in database
Panama Canal drought to delay grain ships well into 2024
Sentiment: bearish
Already in database
Business booms on the sidelines of COP28
bullish
Scotiabank's new CEO must sell investors on his plan to boost Latam profits
Sentiment: bearish
Already in database
Soft-Landing Bets Fuel Canadian Bank Stocks Despite Real Estate Worries
Sentiment: bearish
Already in database
Futures muted as traders brace for inflation data, Fed verdict
Sentiment: bearish
Already in database
Tucker Carlson's streaming service to charge $9 per month
Sentiment: neutral
Already in database
US STOCKS-Futures muted as traders brace for inflation data, Fed verdict
bullish
In 2023, bold EV strategies took a punch from 